# Assignment 1

## Step 1: Uploading and inspecting the data

* `head()` for getting a first few rows of the dataset, looking at the kind of values
* `info()` for getting the description of the data columns
* `value_counts()` for counting the instances of each value in categorical variable (maybe would work for floating-point but results would be rather meaningless).
* `describe()` for per-feature summary statistics

*Questions on interpretation of the dataset from* `describe()`:
1. *Interpreting the values.* Not sure what is meant there. The method just returns per-column (feature) summary statistics.
2. *Meaning of percentiles.* Percentiles tell how much the distribution is skewed based on how far away the median is from each of the quartiles—`households` column is one such example where distribution is skewed to the left (or tail-heavy?).
3. *Handling of missing values.* Documentation says `NaN` by default, which would be skipped in computing statistics (`skipna=True` by default) but would be kept in arrays and propagate through arithmetic calculations. When all values are `NaN` result in automatic statistic calculation is 0.

## Step 2: Splitting the data into training and test sets

* `np.random.permutation(len(data))` for shuffling the indices and then taking `test_indices` and `train_indices` from this accordingly;
  * `data.iloc[train_indices]` for retrieving the data
* `from sklearn.model_selection import train_test_split` for automatically splitting the dataset by describing the fraction of test data
* stratified sampling to get similar train and test distributions if some features are imbalanced—both distributions will be more representative
* binning to limit the number of strata
  * `from sklearn.model_selection import StratifiedShuffleSplit`
  * split based on a categorical attribute: `split.split(df, df["attr"])`
  
## Step 3: Exploring the attributes

* Visualisation to 
  * see most informative attributes 
  * observe correlations with target label using scatter matrix
  * observe the need for normalisation/scaling
  * can also indicate the artifacts in the data that would need cleaning up
* Create extra features

## Step 4: Data preparation and transformations of machine learning algorithms

Required for:
* handling missing values (if any)
  * `from sklearn.impute import SimpleImputer`, `imputer.fit(df)`, `imputer.transform`
* converting attribute values into numerical format (e.g. transform categorical data into numbers)—*is this actually needed? and it doesn't really make sense for a regression task anyway, especially if there is no 'increasing' ordering to categories*
  * numeric value makes the least sense but one-hot encoding using `sklearn.preprocessing.OneHotEncoder`+`sklearn.preprocessing.LabelBinarizer` is better
* scaling or normalising data
* data transformers and pipelines for connecting transformations together with `fit`/`transform`/`fit_transform`
* `FeatureUnion` for connecting together several pipelines (one for each type of feature of categorical/numerical


## Step 5: Implementation, evaluation and fine-tuning of a regression model

* training, testing, cross-validation—the usual.

# Task 2: Experimenting with different steps in the ML pipeline

First implement the base of the ML pipeline.

In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit


np.random.seed(42)

In [ ]:
# loading data
def load_data(housing_path):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_data("housing/")

In [ ]:
# stratifying dataset
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace = True)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
# data preparation and transformations
housing = strat_train_set.drop("median_house_value", axis=1) # inplace=False
housing_labels = strat_train_set["median_house_value"].copy()

def add_features(data):
    # add the transformed features that you found useful before
    data["rooms_per_household"] = data["total_rooms"] / data["households"]
    data["bedrooms_per_household"] = data["total_bedrooms"] / data["households"]
    data["bedrooms_per_rooms"] = data["total_bedrooms"] / data["total_rooms"]
    data["population_per_household"] = data["population"] / data["households"]    

## Dropping less informative features

## Using alternative preprocessing options

In [ ]:
# handling missing values
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)

X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns=housing_num.columns)

In [ ]:
# handling textual and categorical attributes
from sklearn.preprocessing import LabelBinarizer

encoder = LabelBinarizer(sparse_output=True)
housing_cat_1hot = encoder.fit_transform(housing["ocean_proximity"])

In [ ]:
# data transformation
from sklearn.base import TransformerMixin # TransformerMixin allows you to use fit_transform method
from sklearn.base import BaseEstimator
# BaseEstimator allows you to drop *args and **kwargs from you constructor
# and, in addition, allows you to use methods set_params() and get_params()


class CustomLabelBinarizer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelBinarizer(*args, **kwargs)
    def fit(self, X, y=0):
        self.encoder.fit(X)
        return self
    def transform(self, X, y=0):
        return self.encoder.transform(X)
    

# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
    
    
rooms_id, bedrooms_id, population_id, household_id = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_rooms = True): # note no *args and **kwargs used this time
        self.add_bedrooms_per_rooms = add_bedrooms_per_rooms
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_id] / X[:, household_id]
        bedrooms_per_household = X[:, bedrooms_id] / X[:, household_id]
        population_per_household = X[:, population_id] / X[:, household_id]
        if self.add_bedrooms_per_rooms:
            bedrooms_per_rooms = X[:, bedrooms_id] / X[:, rooms_id]
            return np.c_[X, rooms_per_household, bedrooms_per_household, 
                         population_per_household, bedrooms_per_rooms]
        else:
            return np.c_[X, rooms_per_household, bedrooms_per_household, 
                         population_per_household]
        
attr_adder = CombinedAttributesAdder()
housing_extra_attribs = attr_adder.transform(housing.values)

housing_extra_attribs = pd.DataFrame(housing_extra_attribs, columns=list(housing.columns)+
                                     ["rooms_per_household", "bedrooms_per_household", 
                                      "population_per_household", "bedrooms_per_rooms"])

In [ ]:
# feature scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()
housing_tr_scaled = scaler.fit_transform(housing_tr)

In [ ]:
# pipeline
from sklearn.pipeline import Pipeline, FeatureUnion

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('label_binarizer', CustomLabelBinarizer()),
    ])

full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])


housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
# training
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
# loss
from sklearn.metrics import mean_squared_error

# RMSE
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
# cross-validation
from sklearn.model_selection import cross_val_score
    
def analyse_cv(model):   
    scores = cross_val_score(model, housing_prepared, housing_labels,
                             scoring = "neg_mean_squared_error", cv=10)

    # cross-validation expects utility function (greater is better)
    # rather than cost function (lower is better), so the scores returned
    # are negative as they are the opposite of MSE
    sqrt_scores = np.sqrt(-scores) 
    print("Scores:", sqrt_scores)
    print("Mean:", sqrt_scores.mean())
    print("Standard deviation:", sqrt_scores.std())
    
analyse_cv(tree_reg)

## `RandomizedSearchCV`

In [ ]:
# fine-tuning
from sklearn.model_selection import GridSearchCV

# specify the range of hyperparameter values for the grid search to try out 
param_grid = {'n_estimators': [3, 10, 30, 50], 'max_features': [2, 4, 6, 8]}

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring="neg_mean_squared_error")
grid_search.fit(housing_prepared, housing_labels)

grid_search.best_params_

## Adjusting feature importance weights

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_

extra_attribs = ['rooms_per_household', 'bedrooms_per_household', 'population_per_household', 'bedrooms_per_rooms']
cat_one_hot_attribs = ['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

In [ ]:
# evaluating best model
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

final_rmse

## Evaluating performance of simple linear regression

# Task 3: Bike sharing dataset